In [1]:
import pandas as pd 
import re
import requests
import pickle

In [2]:
data = requests.get('https://www.ams.usda.gov/mnreports/gx_gr110.txt').text
data
data2 = data.split('\r\n')
data2

['GX_GR110',
 'Springfield, IL    Thu, Mar 21, 2019    USDA-IL Dept of Ag Market News',
 '',
 'Chicago Terminal Grain Report',
 '',
 'To Arrive Truck and Rail Bids for Grain Delivered to Chicago.  The',
 'following quotations represent Bids ($/ bu) from Terminal Elevators,',
 'Processors, Mills, and Merchandisers after 1:30 p.m. today.',
 '',
 'Grain          Bids        Delivery        Change           Basis       Change',
 '',
 'SRW Wheat  4.4950-4.6150   30 Days         UP 1.75       -17K to -5K     UNCH',
 '',
 'Soybeans   8.6450-8.6650     Spot          UP 4.5       -46K to -44K     UNCH',
 'Soybeans   8.5350-8.6550  15-30 Days       UP 4.5       -57K to -45K     UNCH',
 '',
 'Terminal Elevator Bids',
 'Corn       3.3125-3.5625     Spot          UP 4.75      -45K to -20K     UNCH',
 'Corn       3.4125-3.5625  15-30 Days       UP 4.75      -35K to -20K     UNCH',
 '',
 'Processor Bids',
 'Corn       3.6625-3.8325     Spot      UP 4.75-UP 5.75   -10K to 7K   UNCH-UP 1',
 'Corn      

In [17]:
split = ['Grain','Terminal Elevator Bids', 'Processor Bids', 'Changes']
crops = [ 'SRW Wheat', 'Soybeans', 'Corn']
cols = ['grain', 'bids', 'delivery', 'change1', 'basis', 'change2']

In [18]:
r = re.compile(r'|'.join(split))
reports = re.split(r,data) #splits based on crop
reps = [x.split('\r\n') for x in reports]
#reps = 
reps

[['GX_GR110',
  'Springfield, IL    Thu, Mar 21, 2019    USDA-IL Dept of Ag Market News',
  '',
  'Chicago Terminal '],
 [' Report', '', 'To Arrive Truck and Rail Bids for '],
 [' Delivered to Chicago.  The',
  'following quotations represent Bids ($/ bu) from Terminal Elevators,',
  'Processors, Mills, and Merchandisers after 1:30 p.m. today.',
  '',
  ''],
 ['          Bids        Delivery        Change           Basis       Change',
  '',
  'SRW Wheat  4.4950-4.6150   30 Days         UP 1.75       -17K to -5K     UNCH',
  '',
  'Soybeans   8.6450-8.6650     Spot          UP 4.5       -46K to -44K     UNCH',
  'Soybeans   8.5350-8.6550  15-30 Days       UP 4.5       -57K to -45K     UNCH',
  '',
  ''],
 ['',
  'Corn       3.3125-3.5625     Spot          UP 4.75      -45K to -20K     UNCH',
  'Corn       3.4125-3.5625  15-30 Days       UP 4.75      -35K to -20K     UNCH',
  '',
  ''],
 ['',
  'Corn       3.6625-3.8325     Spot      UP 4.75-UP 5.75   -10K to 7K   UNCH-UP 1',
  'Corn   

In [19]:
list1 = []
for x in reps[:-1]:
    new = [ y for y in x if (y.startswith('SRW Wheat') or y.startswith('Soybeans') or y.startswith('Corn')) ]
    if new !=[]:
        list1.append(new)
list1

[['SRW Wheat  4.4950-4.6150   30 Days         UP 1.75       -17K to -5K     UNCH',
  'Soybeans   8.6450-8.6650     Spot          UP 4.5       -46K to -44K     UNCH',
  'Soybeans   8.5350-8.6550  15-30 Days       UP 4.5       -57K to -45K     UNCH'],
 ['Corn       3.3125-3.5625     Spot          UP 4.75      -45K to -20K     UNCH',
  'Corn       3.4125-3.5625  15-30 Days       UP 4.75      -35K to -20K     UNCH'],
 ['Corn       3.6625-3.8325     Spot      UP 4.75-UP 5.75   -10K to 7K   UNCH-UP 1',
  'Corn       3.6725-3.8425  15-30 Days   UP 4.75-UP 8.75    -9K to 8K   UNCH-UP 4']]

In [20]:
split_type = []
for n,x in enumerate(list1):
    for y in x:
        split_type.append(split[n])
split_type # this give the right column for the data frame

['Grain',
 'Grain',
 'Grain',
 'Terminal Elevator Bids',
 'Terminal Elevator Bids',
 'Processor Bids',
 'Processor Bids']

In [21]:
list3= []
for x in list1:
    for y in x:
        list3.append(re.split(r'\s{2,}',y)) 
        
df3= pd.DataFrame(list3,columns=cols)
df3['location'] = split_type
df3.loc[df3['location']=='Grain','location'] = 'Merchandiser'
df3['date'] = pd.Timestamp.now()
df3

,grain,bids,delivery,change1,basis,change2,location,date
0,SRW Wheat,4.4950-4.6150,30 Days,UP 1.75,-17K to -5K,UNCH,Merchandiser,2019-03-22 14:25:41.089479
1,Soybeans,8.6450-8.6650,Spot,UP 4.5,-46K to -44K,UNCH,Merchandiser,2019-03-22 14:25:41.089479
2,Soybeans,8.5350-8.6550,15-30 Days,UP 4.5,-57K to -45K,UNCH,Merchandiser,2019-03-22 14:25:41.089479
3,Corn,3.3125-3.5625,Spot,UP 4.75,-45K to -20K,UNCH,Terminal Elevator Bids,2019-03-22 14:25:41.089479
4,Corn,3.4125-3.5625,15-30 Days,UP 4.75,-35K to -20K,UNCH,Terminal Elevator Bids,2019-03-22 14:25:41.089479
5,Corn,3.6625-3.8325,Spot,UP 4.75-UP 5.75,-10K to 7K,UNCH-UP 1,Processor Bids,2019-03-22 14:25:41.089479
6,Corn,3.6725-3.8425,15-30 Days,UP 4.75-UP 8.75,-9K to 8K,UNCH-UP 4,Processor Bids,2019-03-22 14:25:41.089479


In [53]:
df3['bids1'] = df3.bids.str.split('-',expand=True)[0]
df3['bids2'] = df3.bids.str.split('-', expand = True)[1]
df3['change1'] = df3['change1'].str.replace('[A-Za-z]|\s',"")
df3['change1_1'] = df3['change1'].str.split('-', expand=True)[0]
df3['change1_2'] = df3['change1'].str.split('-', expand=True)[1]
df3['change2_1'] = df3['change2'].str.split('-', expand=True)[0]
df3['change2_2'] = df3['change2'].str.split('-', expand=True)[1]
df3['basis1'] = df3['basis'].str.split('to', expand=True)[0].str.replace('[A-Za-z]',"")
df3['basis2'] = df3['basis'].str.split('to', expand=True)[1].str.replace('[A-Za-z]',"")
df3['contract'] = df3['basis'].str.split('to', expand=True)[0].str.replace('\d|[+-]',"")
df3

,grain,bids,delivery,change1,basis,change2,location,date,bids1,bids2,change1_1,change1_2,change2_1,change2_2,basis1,basis2,contract
0,SRW Wheat,4.4950-4.6150,30 Days,1.75,-17K to -5K,UNCH,Merchandiser,2019-03-22 14:25:41.089479,4.4950,4.6150,1.75,None,UNCH,None,-17,-5,K
1,Soybeans,8.6450-8.6650,Spot,4.5,-46K to -44K,UNCH,Merchandiser,2019-03-22 14:25:41.089479,8.6450,8.6650,4.5,None,UNCH,None,-46,-44,K
2,Soybeans,8.5350-8.6550,15-30 Days,4.5,-57K to -45K,UNCH,Merchandiser,2019-03-22 14:25:41.089479,8.5350,8.6550,4.5,None,UNCH,None,-57,-45,K
3,Corn,3.3125-3.5625,Spot,4.75,-45K to -20K,UNCH,Terminal Elevator Bids,2019-03-22 14:25:41.089479,3.3125,3.5625,4.75,None,UNCH,None,-45,-20,K
4,Corn,3.4125-3.5625,15-30 Days,4.75,-35K to -20K,UNCH,Terminal Elevator Bids,2019-03-22 14:25:41.089479,3.4125,3.5625,4.75,None,UNCH,None,-35,-20,K
5,Corn,3.6625-3.8325,Spot,4.75-5.75,-10K to 7K,UNCH-UP 1,Processor Bids,2019-03-22 14:25:41.089479,3.6625,3.8325,4.75,5.75,UNCH,UP 1,-10,7,K
6,Corn,3.6725-3.8425,15-30 Days,4.75-8.75,-9K to 8K,UNCH-UP 4,Processor Bids,2019-03-22 14:25:41.089479,3.6725,3.8425,4.75,8.75,UNCH,UP 4,-9,8,K


In [58]:
final = df3[['grain','location','contract','bids1','bids2','basis1','basis1','change1_1', 'change1_2','change2_1','change2_2','date']]
final = final.fillna('NA')
final

,grain,location,contract,bids1,bids2,basis1,basis1,change1_1,change1_2,change2_1,change2_2,date
0,SRW Wheat,Merchandiser,K,4.4950,4.6150,-17,-17,1.75,NA,UNCH,NA,2019-03-22 14:25:41.089479
1,Soybeans,Merchandiser,K,8.6450,8.6650,-46,-46,4.5,NA,UNCH,NA,2019-03-22 14:25:41.089479
2,Soybeans,Merchandiser,K,8.5350,8.6550,-57,-57,4.5,NA,UNCH,NA,2019-03-22 14:25:41.089479
3,Corn,Terminal Elevator Bids,K,3.3125,3.5625,-45,-45,4.75,NA,UNCH,NA,2019-03-22 14:25:41.089479
4,Corn,Terminal Elevator Bids,K,3.4125,3.5625,-35,-35,4.75,NA,UNCH,NA,2019-03-22 14:25:41.089479
5,Corn,Processor Bids,K,3.6625,3.8325,-10,-10,4.75,5.75,UNCH,UP 1,2019-03-22 14:25:41.089479
6,Corn,Processor Bids,K,3.6725,3.8425,-9,-9,4.75,8.75,UNCH,UP 4,2019-03-22 14:25:41.089479


In [59]:
pickle.dump(final,open('gxgr110_frame','wb'))

In [60]:
final.to_csv('gxgr110.csv')